In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [14]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ s', Q = env(s, a)
    ################################################ ~s', ~Q = env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels))
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels))
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a') # max
    ###################################################### ~Q(s,~a)= r # min
    ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
    targetQs = rewards + (gamma * aQs2_logits)
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
    aloss2 += -tf.reduce_mean(aQs_logits) # max
    return actions_logits, aloss, eloss, aloss2

In [15]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [16]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [17]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [18]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [19]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [20]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [21]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:12.0000 R:12.0000 rate:0.0240 aloss:0.7040 eloss:4.6677 aloss2:1.8973 exploreP:0.9988
Episode:1 meanR:11.0000 R:10.0000 rate:0.0200 aloss:0.7038 eloss:4.7047 aloss2:1.8598 exploreP:0.9978
Episode:2 meanR:12.0000 R:14.0000 rate:0.0280 aloss:0.7071 eloss:4.8746 aloss2:1.7666 exploreP:0.9964
Episode:3 meanR:12.7500 R:15.0000 rate:0.0300 aloss:0.7030 eloss:4.7464 aloss2:1.8394 exploreP:0.9950
Episode:4 meanR:14.4000 R:21.0000 rate:0.0420 aloss:0.7010 eloss:4.8579 aloss2:1.7458 exploreP:0.9929
Episode:5 meanR:15.3333 R:20.0000 rate:0.0400 aloss:0.7005 eloss:4.7213 aloss2:1.8318 exploreP:0.9909
Episode:6 meanR:14.4286 R:9.0000 rate:0.0180 aloss:0.6938 eloss:4.8148 aloss2:1.7316 exploreP:0.9901
Episode:7 meanR:14.1250 R:12.0000 rate:0.0240 aloss:0.7041 eloss:4.8642 aloss2:1.7309 exploreP:0.9889
Episode:8 meanR:14.4444 R:17.0000 rate:0.0340 aloss:0.6965 eloss:4.7680 aloss2:1.7622 exploreP:0.9872
Episode:9 meanR:16.1000 R:31.0000 rate:0.0620 aloss:0.7015 eloss:4.8190 aloss2:1.75

Episode:80 meanR:27.1235 R:14.0000 rate:0.0280 aloss:0.6897 eloss:3.8484 aloss2:1.9597 exploreP:0.8047
Episode:81 meanR:27.0854 R:24.0000 rate:0.0480 aloss:0.7150 eloss:3.8642 aloss2:1.9550 exploreP:0.8028
Episode:82 meanR:28.2651 R:125.0000 rate:0.2500 aloss:0.7003 eloss:3.8107 aloss2:1.9873 exploreP:0.7930
Episode:83 meanR:28.8571 R:78.0000 rate:0.1560 aloss:0.6926 eloss:3.7803 aloss2:2.0237 exploreP:0.7869
Episode:84 meanR:29.3176 R:68.0000 rate:0.1360 aloss:0.6984 eloss:3.7735 aloss2:2.0393 exploreP:0.7816
Episode:85 meanR:29.2442 R:23.0000 rate:0.0460 aloss:0.6909 eloss:3.7470 aloss2:2.0611 exploreP:0.7799
Episode:86 meanR:29.1034 R:17.0000 rate:0.0340 aloss:0.6791 eloss:3.7499 aloss2:2.0563 exploreP:0.7785
Episode:87 meanR:29.1364 R:32.0000 rate:0.0640 aloss:0.6875 eloss:3.7609 aloss2:2.0548 exploreP:0.7761
Episode:88 meanR:29.8652 R:94.0000 rate:0.1880 aloss:0.7054 eloss:3.7777 aloss2:2.0496 exploreP:0.7689
Episode:89 meanR:29.6889 R:14.0000 rate:0.0280 aloss:0.7316 eloss:3.8165

Episode:159 meanR:45.4800 R:97.0000 rate:0.1940 aloss:0.7078 eloss:3.4677 aloss2:2.3808 exploreP:0.5556
Episode:160 meanR:45.6000 R:28.0000 rate:0.0560 aloss:0.6851 eloss:3.4465 aloss2:2.3960 exploreP:0.5541
Episode:161 meanR:44.8800 R:15.0000 rate:0.0300 aloss:0.7123 eloss:3.4765 aloss2:2.3768 exploreP:0.5533
Episode:162 meanR:45.1200 R:45.0000 rate:0.0900 aloss:0.6900 eloss:3.4618 aloss2:2.3872 exploreP:0.5508
Episode:163 meanR:44.5800 R:47.0000 rate:0.0940 aloss:0.7243 eloss:3.4813 aloss2:2.3805 exploreP:0.5483
Episode:164 meanR:45.6500 R:128.0000 rate:0.2560 aloss:0.7041 eloss:3.4686 aloss2:2.3973 exploreP:0.5414
Episode:165 meanR:45.4200 R:24.0000 rate:0.0480 aloss:0.7233 eloss:3.4705 aloss2:2.3931 exploreP:0.5402
Episode:166 meanR:45.6300 R:55.0000 rate:0.1100 aloss:0.7159 eloss:3.4884 aloss2:2.3902 exploreP:0.5373
Episode:167 meanR:46.0500 R:52.0000 rate:0.1040 aloss:0.7174 eloss:3.4982 aloss2:2.3872 exploreP:0.5345
Episode:168 meanR:46.5400 R:68.0000 rate:0.1360 aloss:0.7043 el

Episode:238 meanR:55.4300 R:36.0000 rate:0.0720 aloss:0.7937 eloss:3.0526 aloss2:3.8233 exploreP:0.3598
Episode:239 meanR:55.3800 R:23.0000 rate:0.0460 aloss:0.8204 eloss:3.0788 aloss2:3.8214 exploreP:0.3590
Episode:240 meanR:54.9400 R:16.0000 rate:0.0320 aloss:0.8132 eloss:3.0806 aloss2:3.7707 exploreP:0.3584
Episode:241 meanR:54.5400 R:9.0000 rate:0.0180 aloss:0.8858 eloss:3.0737 aloss2:3.7862 exploreP:0.3581
Episode:242 meanR:54.5800 R:51.0000 rate:0.1020 aloss:0.8562 eloss:3.1212 aloss2:3.7772 exploreP:0.3563
Episode:243 meanR:54.4000 R:12.0000 rate:0.0240 aloss:0.7834 eloss:3.0312 aloss2:3.9479 exploreP:0.3559
Episode:244 meanR:54.8300 R:79.0000 rate:0.1580 aloss:0.7872 eloss:3.0282 aloss2:3.9192 exploreP:0.3532
Episode:245 meanR:55.5300 R:87.0000 rate:0.1740 aloss:0.8441 eloss:3.0913 aloss2:3.9234 exploreP:0.3502
Episode:246 meanR:55.7300 R:57.0000 rate:0.1140 aloss:0.7872 eloss:3.0132 aloss2:3.9621 exploreP:0.3483
Episode:247 meanR:55.9100 R:66.0000 rate:0.1320 aloss:0.7806 elos

Episode:317 meanR:88.8800 R:34.0000 rate:0.0680 aloss:0.7288 eloss:2.7122 aloss2:6.2069 exploreP:0.1639
Episode:318 meanR:88.9100 R:40.0000 rate:0.0800 aloss:0.7353 eloss:2.6818 aloss2:6.0950 exploreP:0.1633
Episode:319 meanR:88.9200 R:34.0000 rate:0.0680 aloss:0.7354 eloss:2.7373 aloss2:6.1045 exploreP:0.1628
Episode:320 meanR:89.1100 R:38.0000 rate:0.0760 aloss:0.7805 eloss:2.7823 aloss2:6.1507 exploreP:0.1622
Episode:321 meanR:88.9900 R:34.0000 rate:0.0680 aloss:0.7209 eloss:2.6659 aloss2:6.1642 exploreP:0.1617
Episode:322 meanR:88.9200 R:26.0000 rate:0.0520 aloss:0.7277 eloss:2.7155 aloss2:6.1802 exploreP:0.1613
Episode:323 meanR:88.3200 R:27.0000 rate:0.0540 aloss:0.7507 eloss:2.7578 aloss2:6.2449 exploreP:0.1609
Episode:324 meanR:88.5600 R:36.0000 rate:0.0720 aloss:0.7299 eloss:2.7643 aloss2:6.1368 exploreP:0.1604
Episode:325 meanR:88.6400 R:24.0000 rate:0.0480 aloss:0.7313 eloss:2.8619 aloss2:6.2528 exploreP:0.1600
Episode:326 meanR:88.3800 R:27.0000 rate:0.0540 aloss:0.7445 elo

...

Episode:791 meanR:60.9900 R:56.0000 rate:0.1120 aloss:0.6785 eloss:2.6140 aloss2:6.9513 exploreP:0.0263
Episode:792 meanR:60.9000 R:41.0000 rate:0.0820 aloss:0.6706 eloss:2.5786 aloss2:7.0171 exploreP:0.0263
Episode:793 meanR:61.1200 R:62.0000 rate:0.1240 aloss:0.6586 eloss:2.6350 aloss2:6.9341 exploreP:0.0262
Episode:794 meanR:60.7400 R:54.0000 rate:0.1080 aloss:0.6850 eloss:2.6233 aloss2:6.9074 exploreP:0.0261
Episode:795 meanR:61.0400 R:68.0000 rate:0.1360 aloss:0.6902 eloss:2.6250 aloss2:6.9251 exploreP:0.0260
Episode:796 meanR:60.9600 R:49.0000 rate:0.0980 aloss:0.6895 eloss:2.6486 aloss2:6.9430 exploreP:0.0259
Episode:797 meanR:61.5200 R:101.0000 rate:0.2020 aloss:0.6917 eloss:2.5890 aloss2:6.9294 exploreP:0.0257
Episode:798 meanR:62.0700 R:92.0000 rate:0.1840 aloss:0.6908 eloss:2.6342 aloss2:6.9424 exploreP:0.0256
Episode:799 meanR:63.1000 R:154.0000 rate:0.3080 aloss:0.6714 eloss:2.6164 aloss2:6.9234 exploreP:0.0254
Episode:800 meanR:63.2100 R:50.0000 rate:0.1000 aloss:0.6841 e

Episode:870 meanR:74.0500 R:55.0000 rate:0.1100 aloss:0.6844 eloss:2.6175 aloss2:6.7878 exploreP:0.0188
Episode:871 meanR:74.1600 R:69.0000 rate:0.1380 aloss:0.6850 eloss:2.6268 aloss2:6.8650 exploreP:0.0188
Episode:872 meanR:74.2900 R:54.0000 rate:0.1080 aloss:0.6794 eloss:2.7081 aloss2:6.8828 exploreP:0.0187
Episode:873 meanR:74.4600 R:62.0000 rate:0.1240 aloss:0.6797 eloss:2.6897 aloss2:6.8583 exploreP:0.0187
Episode:874 meanR:74.2500 R:32.0000 rate:0.0640 aloss:0.6893 eloss:2.6611 aloss2:6.8476 exploreP:0.0186
Episode:875 meanR:74.2400 R:61.0000 rate:0.1220 aloss:0.6609 eloss:2.7027 aloss2:6.8653 exploreP:0.0186
Episode:876 meanR:74.4300 R:60.0000 rate:0.1200 aloss:0.6794 eloss:2.7928 aloss2:6.8321 exploreP:0.0185
Episode:877 meanR:73.3900 R:50.0000 rate:0.1000 aloss:0.6900 eloss:2.6651 aloss2:6.8391 exploreP:0.0185
Episode:878 meanR:73.4300 R:68.0000 rate:0.1360 aloss:0.6464 eloss:2.6636 aloss2:6.8585 exploreP:0.0184
Episode:879 meanR:73.4400 R:60.0000 rate:0.1200 aloss:0.7069 elo

Episode:949 meanR:90.1200 R:109.0000 rate:0.2180 aloss:0.6458 eloss:2.7869 aloss2:6.7344 exploreP:0.0142
Episode:950 meanR:89.8700 R:57.0000 rate:0.1140 aloss:0.6235 eloss:2.7588 aloss2:6.7829 exploreP:0.0141
Episode:951 meanR:89.7600 R:46.0000 rate:0.0920 aloss:0.6325 eloss:2.7426 aloss2:6.7457 exploreP:0.0141
Episode:952 meanR:90.0200 R:68.0000 rate:0.1360 aloss:0.6470 eloss:2.7586 aloss2:6.7796 exploreP:0.0141
Episode:953 meanR:90.4600 R:98.0000 rate:0.1960 aloss:0.6513 eloss:2.7692 aloss2:6.7294 exploreP:0.0141
Episode:954 meanR:90.3600 R:30.0000 rate:0.0600 aloss:0.6703 eloss:2.6858 aloss2:6.6595 exploreP:0.0140
Episode:955 meanR:90.7500 R:83.0000 rate:0.1660 aloss:0.6245 eloss:2.7547 aloss2:6.7408 exploreP:0.0140
Episode:956 meanR:94.2200 R:399.0000 rate:0.7980 aloss:0.6438 eloss:2.7372 aloss2:6.7167 exploreP:0.0139
Episode:957 meanR:94.2600 R:70.0000 rate:0.1400 aloss:0.6602 eloss:2.6550 aloss2:6.7445 exploreP:0.0138
Episode:958 meanR:94.5500 R:75.0000 rate:0.1500 aloss:0.6504 e

Episode:1028 meanR:99.1400 R:55.0000 rate:0.1100 aloss:0.6024 eloss:2.7921 aloss2:6.5300 exploreP:0.0121
Episode:1029 meanR:99.0700 R:70.0000 rate:0.1400 aloss:0.6160 eloss:2.7850 aloss2:6.4993 exploreP:0.0120
Episode:1030 meanR:100.9000 R:246.0000 rate:0.4920 aloss:0.6138 eloss:2.7703 aloss2:6.5410 exploreP:0.0120
Episode:1031 meanR:100.7300 R:52.0000 rate:0.1040 aloss:0.5556 eloss:2.8147 aloss2:6.5280 exploreP:0.0120
Episode:1032 meanR:99.7800 R:135.0000 rate:0.2700 aloss:0.6014 eloss:2.7956 aloss2:6.5396 exploreP:0.0120
Episode:1033 meanR:98.5900 R:69.0000 rate:0.1380 aloss:0.5856 eloss:2.7505 aloss2:6.5707 exploreP:0.0119
Episode:1034 meanR:98.3100 R:53.0000 rate:0.1060 aloss:0.6538 eloss:2.8209 aloss2:6.5593 exploreP:0.0119
Episode:1035 meanR:96.3700 R:55.0000 rate:0.1100 aloss:0.5861 eloss:2.8241 aloss2:6.4871 exploreP:0.0119
Episode:1036 meanR:96.4400 R:68.0000 rate:0.1360 aloss:0.5906 eloss:2.8618 aloss2:6.4703 exploreP:0.0119
Episode:1037 meanR:96.2400 R:67.0000 rate:0.1340 al

Episode:1106 meanR:80.8000 R:50.0000 rate:0.1000 aloss:0.5196 eloss:2.8584 aloss2:6.5277 exploreP:0.0111
Episode:1107 meanR:80.0100 R:63.0000 rate:0.1260 aloss:0.5575 eloss:2.7698 aloss2:6.4176 exploreP:0.0111
Episode:1108 meanR:80.0600 R:71.0000 rate:0.1420 aloss:0.6235 eloss:2.7962 aloss2:6.3897 exploreP:0.0111
Episode:1109 meanR:80.2500 R:68.0000 rate:0.1360 aloss:0.5782 eloss:2.7559 aloss2:6.5131 exploreP:0.0111
Episode:1110 meanR:79.9800 R:67.0000 rate:0.1340 aloss:0.5509 eloss:2.7604 aloss2:6.4978 exploreP:0.0111
Episode:1111 meanR:80.1900 R:65.0000 rate:0.1300 aloss:0.5476 eloss:2.8617 aloss2:6.4914 exploreP:0.0111
Episode:1112 meanR:80.8400 R:140.0000 rate:0.2800 aloss:0.5913 eloss:2.8423 aloss2:6.4443 exploreP:0.0111
Episode:1113 meanR:79.6600 R:75.0000 rate:0.1500 aloss:0.5471 eloss:2.8261 aloss2:6.4352 exploreP:0.0110
Episode:1114 meanR:79.3800 R:53.0000 rate:0.1060 aloss:0.5859 eloss:2.7947 aloss2:6.4619 exploreP:0.0110
Episode:1115 meanR:78.2400 R:59.0000 rate:0.1180 aloss

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
